In [1]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context

from mindnlp.transforms import PadTransform
from mindnlp.transforms.tokenizers import BertTokenizer

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

[ERROR] ME(37624:140504709490496,MainProcess):2023-05-07-01:49:49.938.065 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcu*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
[ERROR] ME(37624:140504709490496,MainProcess):2023-05-07-01:49:49.958.065 [mindspore/run_check/_check_version.py:226] Cuda ['10.1', '11.1', '11.6'] version(libcudnn*.so need by mindspore-gpu) is not found. Please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, or check whether the CUDA version in wheel package and the CUDA runtime in current device matches. Please refer to the installation guidelines: https://www.mindspore.cn/install
/data/miniconda3/envs/mindspore/lib/python3.7/site-packages/mindnlp/utils/download.py:26: Tqd

In [2]:
# prepare dataset
class SentimentDataset:
    """Sentiment Dataset"""

    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

In [3]:
# download dataset
!wget https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz -O emotion_detection.tar.gz
!tar xvf emotion_detection.tar.gz

--2023-05-07 01:49:51--  https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz
Resolving baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)... 36.110.192.138, 36.110.192.178, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to baidu-nlp.bj.bcebos.com (baidu-nlp.bj.bcebos.com)|36.110.192.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1710581 (1.6M) [application/x-gzip]
Saving to: ‘emotion_detection.tar.gz’

emotion_detection.t 100%[===================>]   1.63M  1.27MB/s    in 1.3s    

2023-05-07 01:49:53 (1.27 MB/s) - ‘emotion_detection.tar.gz’ saved [1710581/1710581]

data/
data/test.tsv
data/infer.tsv
data/dev.tsv
data/train.tsv
data/vocab.txt


In [4]:
def process_dataset(source, tokenizer, pad_value, max_seq_len=64, batch_size=32, shuffle=True):
    column_names = ["label", "text_a"]
    rename_columns = ["label", "input_ids"]
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    pad_op = PadTransform(max_seq_len, pad_value=pad_value)
    type_cast_op = transforms.TypeCast(mindspore.int32)
    
    # map dataset
    dataset = dataset.map(operations=[tokenizer, pad_op], input_columns="text_a")
    dataset = dataset.map(operations=[type_cast_op], input_columns="label")
    # rename dataset
    dataset = dataset.rename(input_columns=column_names, output_columns=rename_columns)
    # batch dataset
    dataset = dataset.batch(batch_size)

    return dataset

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
pad_value = tokenizer.token_to_id('[PAD]')

In [6]:
dataset_train = process_dataset(SentimentDataset("data/train.tsv"), tokenizer, pad_value)
dataset_val = process_dataset(SentimentDataset("data/dev.tsv"), tokenizer, pad_value)
dataset_test = process_dataset(SentimentDataset("data/test.tsv"), tokenizer, pad_value, shuffle=False)

In [7]:
from mindnlp.models import BertForSequenceClassification
from mindnlp._legacy.amp import auto_mixed_precision

# set bert config and define parameters for training
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)
model = auto_mixed_precision(model, 'O1')

loss = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

metric = Accuracy()

# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_emotect', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_emotect_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=5, loss_fn=loss, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=True)

[WARNING] ME(37624:140504709490496,MainProcess):2023-05-07-01:50:12.118.592 [mindspore/train/serialization.py:1088] For 'load_param_into_net', 2 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(37624:140504709490496,MainProcess):2023-05-07-01:50:12.121.378 [mindspore/train/serialization.py:1090] classifier.weight is not loaded.
[WARNING] ME(37624:140504709490496,MainProcess):2023-05-07-01:50:12.121.932 [mindspore/train/serialization.py:1090] classifier.bias is not loaded.


In [8]:
# start training
trainer.run('label')

The train will start from the checkpoint saved in 'checkpoint'.


  0%|          | 0/302 [00:00<?, ?it/s]

Checkpoint: 'bert_emotect_epoch_0.ckpt' has been saved in epoch: 0.


  0%|          | 0/34 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.9101851851851852}
---------------Best Model: 'bert_emotect_best.ckpt' has been saved in epoch: 0.---------------


...

Evaluate Score: {'Accuracy': 0.9629629629629629}


  0%|          | 0/302 [00:00<?, ?it/s]

The maximum number of stored checkpoints has been reached.
Checkpoint: 'bert_emotect_epoch_4.ckpt' has been saved in epoch: 4.


  0%|          | 0/34 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.9675925925925926}
---------------Best Model: 'bert_emotect_best.ckpt' has been saved in epoch: 4.---------------
Loading best model from 'checkpoint' with '['Accuracy']': [0.9675925925925926]...
---------------The model is already load the best model from 'bert_emotect_best.ckpt'.---------------


In [9]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="label")

  0%|          | 0/33 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.8967181467181468}


In [10]:
dataset_infer = SentimentDataset("data/infer.tsv")

In [35]:
def predict(text, label=None):
    label_map = {0: "消极", 1: "中性", 2: "积极"}

    text_tokenized = Tensor([tokenizer.encode(text).ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)

In [36]:
from mindspore import Tensor

for label, text in dataset_infer:
    predict(text, label)

inputs: '我 要 客观', predict: '中性' , label: '中性'
inputs: '靠 你 真是 说 废话 吗', predict: '消极' , label: '消极'
inputs: '口嗅 会', predict: '中性' , label: '中性'
inputs: '每次 是 表妹 带 窝 飞 因为 窝路痴', predict: '中性' , label: '中性'
inputs: '别说 废话 我 问 你 个 问题', predict: '消极' , label: '消极'
inputs: '4967 是 新加坡 那 家 银行', predict: '中性' , label: '中性'
inputs: '是 我 喜欢 兔子', predict: '积极' , label: '积极'
inputs: '你 写 过 黄山 奇石 吗', predict: '中性' , label: '中性'
inputs: '一个一个 慢慢来', predict: '中性' , label: '中性'
inputs: '我 玩 过 这个 一点 都 不 好玩', predict: '消极' , label: '消极'
inputs: '网上 开发 女孩 的 QQ', predict: '中性' , label: '中性'
inputs: '背 你 猜 对 了', predict: '中性' , label: '中性'
inputs: '我 讨厌 你 ， 哼哼 哼 。 。', predict: '消极' , label: '消极'


In [40]:
predict("家人们咱就是说一整个无语住了 绝绝子叠buff")

inputs: '家人们咱就是说一整个无语住了 绝绝子叠buff', predict: '消极'
